<a href="https://colab.research.google.com/github/vadhri/distibuted-optimization/blob/main/graph-based-topology/laplacian_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
!pip install -q igraph

In [102]:
import igraph as ig
import numpy as np

N = 1000

# construct a full graph
g=ig.Graph.Full(N)

A = g.get_adjacency()
# construct diagonal matrrix from degree
D = np.diag(g.degree())

# construct Laplacian
L = D - A

x = [10,]*N
b = np.dot(L,x)
print(' avg = ', np.average(b))

 avg =  0.0


In [103]:
x = np.random.uniform(0,1000,N)
#compute median and avg of x
alpha = (1/N)
print ('median = ', np.median(x), 'avg = ', np.average(x))
for iter in range(100):
  x = x - (alpha * L @ x)
  # check if values of x are in tol limits to each otherr
  tol = x[0]
  conv = np.all([np.isclose(tol, i) for i in x])
  if conv == True:
    print ('converged at iter = ', iter)
    print (x)
    break

median =  523.0414856775477 avg =  505.4024123250997
converged at iter =  0
[505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233
 505.40241233 505.40241233 505.40241233 505.40241233 505.40241233


In [104]:
%%writefile laplacian.c
#include <igraph.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 10000
#define MAX_ITER 100
#define TOL 1e-4
#define ALPHA (1/N)

int main() {
    igraph_t g;
    igraph_matrix_t A, L;
    igraph_real_t *x = calloc(N, sizeof(igraph_real_t));
    igraph_real_t *x_next = calloc(N, sizeof(igraph_real_t));

    // Create a fully connected undirected graph
    igraph_full(&g, N, IGRAPH_UNDIRECTED, IGRAPH_NO_LOOPS);

    // Initialize x with random values between 100 and 500
    for (int i = 0; i < N; ++i)
        x[i] = (rand() % 400) + 100;

    // Calculate initial average
    igraph_real_t avg = 0.0;
    for (int i = 0; i < N; ++i)
        avg += x[i];
    avg /= N;
    printf("Average of x: %.4f\n", avg);

    // Compute adjacency matrix A
    igraph_matrix_init(&A, N, N);
    igraph_get_adjacency(&g, &A, IGRAPH_GET_ADJACENCY_BOTH, /*loops=*/0);

    // Compute Laplacian L = D - A
    igraph_matrix_init(&L, N, N);
    for (int i = 0; i < N; ++i) {
        igraph_real_t degree = 0.0;
        for (int j = 0; j < N; ++j) {
            degree += MATRIX(A, i, j);
        }
        for (int j = 0; j < N; ++j) {
            if (i == j)
                MATRIX(L, i, j) = degree;
            else
                MATRIX(L, i, j) = -MATRIX(A, i, j);
        }
    }

    // Apply Laplacian dynamics
    int iter = 0;
    igraph_real_t diff = INFINITY;

    while (iter < MAX_ITER && diff > TOL) {
        diff = 0.0;

        for (int i = 0; i < N; ++i) {
            igraph_real_t lap = 0.0;
            for (int j = 0; j < N; ++j) {
                lap += MATRIX(L, i, j) * x[j];
            }
            x_next[i] = x[i] - ALPHA * lap;
            diff += fabs(x_next[i] - x[i]);
        }

        // Swap x and x_next
        igraph_real_t *temp = x;
        x = x_next;
        x_next = temp;

        if (iter % 10 == 0)
            printf("Iteration %d, diff = %.6f\n", iter, diff / N);

        ++iter;
    }

    // Print first 10 values
    for (int i = 0; i < 10; ++i)
        printf("%.4f ", x[i]);
    printf("\n");

    printf("Converged in %d iterations. Consensus value: %.4f\n", iter, x[0]);

    // Cleanup
    igraph_matrix_destroy(&A);
    igraph_matrix_destroy(&L);
    igraph_destroy(&g);
    free(x);
    free(x_next);

    return 0;
}



Overwriting laplacian.c


In [105]:
!ls /usr/include/igraph/igraph.h


/usr/include/igraph/igraph.h


In [106]:
%%bash
apt-get update
apt-get install -y libigraph-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
libigraph-dev is already the newest version (0.9.6+ds-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [107]:
!gcc laplacian.c -I/usr/include/igraph -L/usr/lib/x86_64-linux-gnu -ligraph -lm -o laplacian
!time ./laplacian

Average of x: 298.4811
Iteration 0, diff = 0.000000
283.0000 186.0000 477.0000 215.0000 293.0000 435.0000 286.0000 192.0000 349.0000 321.0000 
Converged in 1 iterations. Consensus value: 283.0000

real	0m30.383s
user	0m27.759s
sys	0m2.480s


In [108]:
%%writefile laplacian_cuda.cu
#include <igraph.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 15000         // Adjust to 10000 if your GPU has enough memory
#define MAX_ITER 1000
#define TOL 1e-3
#define ALPHA 0.0001

// CUDA kernel for Laplacian update: x_next = x - α * Lx
__global__ void laplacian_update(double* x, double* x_next, double* L, int n, double alpha) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        double Lx_i = 0.0;
        for (int j = 0; j < n; ++j) {
            Lx_i += L[i * n + j] * x[j];
        }
        x_next[i] = x[i] - alpha * Lx_i;
    }
}

int main() {
    igraph_t g;
    igraph_matrix_t adj;
    igraph_matrix_init(&adj, 0, 0);
    igraph_full(&g, N, IGRAPH_UNDIRECTED, IGRAPH_NO_LOOPS);
    igraph_get_adjacency(&g, &adj, IGRAPH_GET_ADJACENCY_BOTH, /*loops=*/0);

    // Compute degree matrix D
    igraph_vector_t degrees;
    igraph_vector_init(&degrees, 0);
    igraph_degree(&g, &degrees, igraph_vss_all(), IGRAPH_ALL, IGRAPH_NO_LOOPS);

    // Build Laplacian matrix L = D - A
    double* L = (double*)malloc(N * N * sizeof(double));
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            if (i == j) {
                L[i * N + j] = VECTOR(degrees)[i];
            } else {
                L[i * N + j] = -MATRIX(adj, i, j);
            }
        }
    }

    // Initialize x with random values
    double* x_host = (double*)malloc(N * sizeof(double));
    double* x_next_host = (double*)malloc(N * sizeof(double));
    for (int i = 0; i < N; ++i)
        x_host[i] = (rand() % 400) + 100;

    // compute avg of x_host values
    double avg = 0.0;
    for (int i = 0; i < N; ++i)
        avg += x_host[i];
    avg /= N;
    printf("Average of x: %.4f\n", avg);

    // CUDA device memory
    double *x_dev, *x_next_dev, *L_dev;
    cudaMalloc(&x_dev, N * sizeof(double));
    cudaMalloc(&x_next_dev, N * sizeof(double));
    cudaMalloc(&L_dev, N * N * sizeof(double));

    // Copy data to device
    cudaMemcpy(x_dev, x_host, N * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(L_dev, L, N * N * sizeof(double), cudaMemcpyHostToDevice);

    // Kernel launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Consensus iteration loop
    int iter = 0;
    double diff = INFINITY;
    while (iter < MAX_ITER && diff > TOL) {
        laplacian_update<<<blocksPerGrid, threadsPerBlock>>>(x_dev, x_next_dev, L_dev, N, ALPHA);
        cudaDeviceSynchronize();

        // Copy back and compute diff
        cudaMemcpy(x_next_host, x_next_dev, N * sizeof(double), cudaMemcpyDeviceToHost);
        diff = 0.0;
        for (int i = 0; i < N; ++i)
            diff += fabs(x_next_host[i] - x_host[i]);

        // Swap pointers
        double* tmp = x_host;
        x_host = x_next_host;
        x_next_host = tmp;
        cudaMemcpy(x_dev, x_host, N * sizeof(double), cudaMemcpyHostToDevice);

        if (iter % 10 == 0)
            printf("Iter %d: avg diff = %.6f\n", iter, diff / N);
        iter++;
    }

    // Final output
    printf("Converged in %d iterations. First 10 values of x:\n", iter);
    for (int i = 0; i < 10; ++i)
        printf("%.4f ", x_host[i]);
    printf("\n");

    // prinnt any cuda errorrs
    cudaError_t cudaErr = cudaGetLastError();
    if (cudaErr != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(cudaErr));
    }

    // Free
    igraph_destroy(&g);
    igraph_vector_destroy(&degrees);
    igraph_matrix_destroy(&adj);
    cudaFree(x_dev);
    cudaFree(x_next_dev);
    cudaFree(L_dev);
    free(x_host);
    free(x_next_host);
    free(L);
    return 0;
}



Overwriting laplacian_cuda.cu


In [109]:
!nvcc laplacian_cuda.cu -ligraph -o laplacian_cuda -I/usr/include/igraph -L/usr/lib/x86_64-linux-gnu -ligraph -lm -gencode arch=compute_75,code=sm_75
!time ./laplacian_cuda


Average of x: 298.9270
Iter 0: avg diff = 150.782169
Iter 10: avg diff = 0.147248
Iter 20: avg diff = 0.000144
Iter 30: avg diff = 0.000000
Converged in 33 iterations. First 10 values of x:
298.9270 298.9270 298.9270 298.9270 298.9270 298.9270 298.9270 298.9270 298.9270 298.9270 

real	0m53.902s
user	0m47.864s
sys	0m5.415s
